In [22]:
import os
from dotenv import find_dotenv, load_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [23]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [24]:
text= """
Mojo combines the usability of Python with the performance of C, unlocking unparalleled programmability \
of AI hardware and extensibility of AI models.
Mojo is a new programming language that bridges the gap between research and production \ 
by combining the best of Python syntax with systems programming and metaprogramming.
With Mojo, you can write portable code that’s faster than C and seamlessly inter-op with the Python ecosystem.
When we started Modular, we had no intention of building a new programming language. \
But as we were building our platform with the intent to unify the world’s ML/AI infrastructure, \
we realized that programming across the entire stack was too complicated. Plus, we were writing a \
lot of MLIR by hand and not having a good time.
And although accelerators are important, one of the most prevalent and sometimes overlooked "accelerators" \
is the host CPU. Nowadays, CPUs have lots of tensor-core-like accelerator blocks and other AI acceleration \
units, but they also serve as the “fallback” for operations that specialized accelerators don’t handle, \
such as data loading, pre- and post-processing, and integrations with foreign systems. \
"""

messages = [
    SystemMessage(content='You are an expert copywriter with expertize in summarizing documents'),
    HumanMessage(content=f'Please provide a short and concise summary of the following text:\n TEXT: {text}')
]

llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo')

In [25]:
summary_output = llm(messages)

In [26]:
print(summary_output.content)

Mojo is a new programming language that combines the usability of Python with the performance of C. It aims to bridge the gap between research and production in the field of AI by offering portable code that is faster than C and seamlessly integrates with the Python ecosystem. Mojo was developed to simplify programming across the entire ML/AI infrastructure and to address the importance of host CPUs in AI acceleration.


### Summarising Using Prompt Templates

In [27]:
from langchain import PromptTemplate
from langchain.chains import LLMChain

In [28]:
template = '''
Write a concise and short summary of the following text:
TEXT: `{text}`
Translate the summary to {language}.
'''
prompt = PromptTemplate(
    input_variables=['text', 'language'],
    template=template
)

In [29]:
llm.get_num_tokens(prompt.format(text=text, language='English'))

251

In [30]:
chain = LLMChain(llm=llm, prompt=prompt)
summary = chain.run({'text': text, 'language': 'deutsche'})

In [31]:
print(summary)

Mojo ist eine neue Programmiersprache, die die Benutzerfreundlichkeit von Python mit der Leistung von C kombiniert und dadurch eine beispiellose Programmierbarkeit von KI-Hardware und Erweiterbarkeit von KI-Modellen ermöglicht. Mit Mojo können Sie portablen Code schreiben, der schneller als C ist und nahtlos mit dem Python-Ökosystem interagiert. Mojo wurde entwickelt, um die Lücke zwischen Forschung und Produktion zu schließen und vereinfacht das Programmieren über den gesamten Stack hinweg. CPUs spielen eine wichtige Rolle als "Beschleuniger" und dienen als Backup für Operationen, die spezialisierte Beschleuniger nicht handhaben können.


### Summarising with StuffDocumentChain

In [32]:
from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document

In [33]:
with open('files/sj.txt', encoding='utf-8') as f:
    text = f.read()

# text

docs = [Document(page_content=text)]
llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo')

In [34]:
template = '''Write a concise and short summary of the following text.
TEXT: `{text}`
'''
prompt = PromptTemplate(
    input_variables=['text'],
    template=template
)

In [35]:
chain = load_summarize_chain(
    llm,
    chain_type='stuff',
    prompt=prompt,
    verbose=False
)
output_summary = chain.run(docs)

In [36]:
print(output_summary)

The speaker, Steve Jobs, shares three stories from his life during a commencement speech. The first story is about dropping out of college and how it led him to take a calligraphy class, which later influenced the design of the Macintosh computer. The second story is about getting fired from Apple and how it allowed him to start over and eventually create successful companies like NeXT and Pixar. The third story is about facing death when he was diagnosed with cancer and how it made him realize the importance of following his heart and not wasting time. He ends the speech by encouraging the graduates to stay hungry and stay foolish.


### Summarizing Large Documents Using map_reduce

In [37]:
from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [38]:
with open('files/sj.txt', encoding='utf-8') as f:
    text = f.read()

llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo')

In [39]:
llm.get_num_tokens(text)

2653

In [40]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=50)
chunks = text_splitter.create_documents([text])

In [41]:
len(chunks)

2

In [42]:
chain = load_summarize_chain(
    llm,
    chain_type='map_reduce',
    verbose=False
)
output_summary = chain.run(chunks)

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-orrPVFAH2xlCHV5RCW5V4EQC on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}

In [ ]:
print(output_summary)

In [43]:
chain.llm_chain.prompt.template

'Write a concise summary of the following:\n\n\n"{text}"\n\n\nCONCISE SUMMARY:'

In [ ]:
chain.combine_document_chain.llm_chain.prompt.template

### map_reduce with Custom Prompts

In [ ]:
map_prompt = '''
Write a short and concise summary of the following:
Text: `{text}`
CONCISE SUMMARY:
'''
map_prompt_template = PromptTemplate(
    input_variables=['text'],
    template=map_prompt
)

In [ ]:
combine_prompt = '''
Write a concise summary of the following text that covers the key points.
Add a title to the summary.
Start your summary with an INTRODUCTION PARAGRAPH that gives an overview of the topic FOLLOWED
by BULLET POINTS if possible AND end the summary with a CONCLUSION PHRASE.
Text: `{text}`
'''
combine_prompt_template = PromptTemplate(template=combine_prompt, input_variables=['text'])

In [ ]:
summary_chain = load_summarize_chain(
    llm=llm,
    chain_type='map_reduce',
    map_prompt=map_prompt_template,
    combine_prompt=combine_prompt_template,
    verbose=False
)
output = summary_chain.run(chunks)

In [ ]:
print(output)

### Summarizing Using the refine Chain

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredPDFLoader

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

In [ ]:
pip install unstructured -q

In [ ]:
pip install pdf2image

In [ ]:
loader = UnstructuredPDFLoader('files/attention_is_all_you_need.pdf')
data = loader.load()

In [ ]:
print(data[0].page_content)

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=100)
chunks = text_splitter.split_documents(data)

In [ ]:
len(chunks)

In [ ]:
llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo')

In [ ]:
def print_embedding_cost(texts):
    import tiktoken
    enc = tiktoken.encoding_for_model('gpt-3.5-turbo')
    total_tokens = sum([len(enc.encode(page.page_content)) for page in texts])
    print(f'Total Tokens: {total_tokens}')
    print(f'Embedding Cost in USD: {total_tokens / 1000 * 0.002:.6f}')
    
    
print_embedding_cost(chunks)

In [ ]:
chain = load_summarize_chain(
    llm=llm,
    chain_type='refine',
    verbose=True
)
output_summary = chain.run(chunks)

In [ ]:
print(output_summary)

### refine With Custom Prompts

In [ ]:
prompt_template = """Write a concise summary of the following extracting the key information:
Text: `{text}`
CONCISE SUMMARY:"""
initial_prompt = PromptTemplate(template=prompt_template, input_variables=['text'])

refine_template = '''
    Your job is to produce a final summary.
    I have provided an existing summary up to a certain point: {existing_answer}.
    Please refine the existing summary with some more context below.
    ------------
    {text}
    ------------
    Start the final summary with an INTRODUCTION PARAGRAPH that gives an overview of the topic FOLLOWED
    by BULLET POINTS if possible AND end the summary with a CONCLUSION PHRASE.
    
'''
refine_prompt = PromptTemplate(
    template=refine_template,
    input_variables=['existing_answer', 'text']
)

In [ ]:
chain = load_summarize_chain(
    llm=llm,
    chain_type='refine',
    question_prompt=initial_prompt,
    refine_prompt=refine_prompt,
    return_intermediate_steps=False
    
)
output_summary = chain.run(chunks)

In [ ]:
print(output_summary)

### Summarizing Using LangChain Agents

In [46]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, Tool
from langchain.utilities import WikipediaAPIWrapper

In [45]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [47]:
llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo')
wikipedia = WikipediaAPIWrapper()

In [48]:
tools = [
    Tool(
        name="Wikipedia", 
        func=wikipedia.run,
        description="Useful for when you need to get information from wikipedia about a single topic"
    )
]

In [49]:
agent_executor = initialize_agent(tools, llm, agent='zero-shot-react-description', verbose=True)

In [50]:
output = agent_executor.run('Can you please provide a short summary of George Washington?')



> Entering new AgentExecutor chain...
I should use Wikipedia to find a short summary of George Washington.
Action: Wikipedia
Action Input: George Washington
Observation: Page: George Washington
Summary: George Washington (February 22, 1732 – December 14, 1799) was an American military officer, statesman, and Founding Father who served as the first president of the United States from 1789 to 1797. Appointed by the Second Continental Congress as commander of the Continental Army in June 1775, Washington led Patriot forces to victory in the American Revolutionary War and then served as president of the Constitutional Convention in 1787, which drafted and ratified the Constitution of the United States and established the American federal government. Washington has thus been called the "Father of his Country".
Washington's first public office, from 1749 to 1750, was as surveyor of Culpeper County in the Colony of Virginia. He subsequently received military training and was assigned comman

In [51]:
print(output)

George Washington (February 22, 1732 – December 14, 1799) was an American military officer, statesman, and Founding Father who served as the first president of the United States from 1789 to 1797. He played a crucial role in the American Revolutionary War, led the drafting and ratification of the Constitution, and implemented a strong national government as president. He is considered one of the greatest U.S. presidents.
